In [28]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np  # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ai-comp/sample_submission.csv
/kaggle/input/ai-comp/train.csv
/kaggle/input/ai-comp/test.csv


In [29]:
# load data
train = pd.read_csv('/kaggle/input/ai-comp/train.csv')
test = pd.read_csv('/kaggle/input/ai-comp/test.csv')
sub = pd.read_csv('/kaggle/input/ai-comp/sample_submission.csv')

In [30]:
# remove unnecessary column
train = train.drop(columns=['ID'])
test = test.drop(columns=['ID'])

In [31]:
train

,first_party,second_party,facts,first_party_winner
0,Phil A. St. Amant,Herman A. Thompson,"On June 27, 1962, Phil St. Amant, a candidate ...",1
1,Stephen Duncan,Lawrence Owens,Ramon Nelson was riding his bike when he suffe...,0
2,Billy Joe Magwood,"Tony Patterson, Warden, et al.",An Alabama state court convicted Billy Joe Mag...,1
3,Linkletter,Walker,Victor Linkletter was convicted in state court...,0
4,William Earl Fikes,Alabama,"On April 24, 1953 in Selma, Alabama, an intrud...",1
...,...,...,...,...
2473,"HollyFrontier Cheyenne Refining, LLC, et al.","Renewable Fuels Association, et al.",Congress amended the Clean Air Act through the...,1
2474,"Grupo Mexicano de Desarrollo, S. A.","Alliance Bond Fund, Inc.","Alliance Bond Fund, Inc., an investment fund, ...",1
2475,Peguero,United States,"In 1992, the District Court sentenced Manuel D...",0
2476,Immigration and Naturalization Service,St. Cyr,"On March 8, 1996, Enrico St. Cyr, a lawful per...",0


In [32]:
test

,first_party,second_party,facts
0,Salerno,United States,The 1984 Bail Reform Act allowed the federal c...
1,Milberg Weiss Bershad Hynes and Lerach,"Lexecon, Inc.",Lexecon Inc. was a defendant in a class action...
2,No. 07-582\t Title: \t Federal Communications ...,"Fox Television Stations, Inc., et al.","In 2002 and 2003, Fox Television Stations broa..."
3,Harold Kaufman,United States,During his trial for armed robbery of a federa...
4,Berger,Hanlon,"In 1993, a magistrate judge issued a warrant a..."
...,...,...,...
1235,"Haitian Centers Council, Inc., et al.","Chris Sale, Acting Commissioner, Immigration A...",According to Executive Order No. 12807 signed ...
1236,Whitman,"American Trucking Associations, Inc.",Section 109(a) of the Clean Air Act (CAA) requ...
1237,Linda A. Matteo and John J. Madigan,William G. Barr,Linda Matteo and John Madigan created a plan f...
1238,Washington State Apple Advertising Commission,Hunt,"In 1972, the North Carolina Board of Agricultu..."


# FEATRUE ENGINEERING

In [33]:
# sampling
from sklearn.utils import resample

subset_0 = train[train["first_party_winner"] == 0]
subset_1 = train[train["first_party_winner"] == 1]

# downsampling
subset_1_downsampled = resample(subset_1,
                                replace=False,
                                n_samples=829,
                                random_state=42)

train = pd.concat([subset_0, subset_1_downsampled])

## upsampling
# subset_0_upsampled = resample(subset_0,
#                               replace=True,
#                               n_samples=1649,
#                               random_state=42)

# train = pd.concat([subset_0_upsampled, subset_1])

In [34]:
# nltk
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

porter = PorterStemmer()

def stem_text_func(text):
    token_words = word_tokenize(text)
    stem_sentence = []
    for word in token_words:
        stem_sentence.append(porter.stem(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [35]:
!pip install -q stop_words

In [36]:
from stop_words import get_stop_words

# stop words language list
languages = [
   'English'
]
my_stop_word_list = []
for i in languages:
    my_stop_word_list.append(get_stop_words(i.lower()))
my_stop_word_list = sum(my_stop_word_list, [])
my_stop_word_list = my_stop_word_list + ['paperbook', 'hardcover']

In [37]:
# contractions from
# https://gist.github.com/nealrs/96342d8231b75cf4bb82
contractions = {
  "ain't": "am not",
  "aren't": "are not",
  "can't": "cannot",
  "can't've": "cannot have",
  "'cause": "because",
  "could've": "could have",
  "couldn't": "could not",
  "couldn't've": "could not have",
  "didn't": "did not",
  "doesn't": "does not",
  "don't": "do not",
  "hadn't": "had not",
  "hadn't've": "had not have",
  "hasn't": "has not",
  "haven't": "have not",
  "he'd": "he would",
  "he'd've": "he would have",
  "he'll": "he will",
  "he'll've": "he will have",
  "he's": "he is",
  "how'd": "how did",
  "how'd'y": "how do you",
  "how'll": "how will",
  "how's": "how is",
  "I'd": "I would",
  "I'd've": "I would have",
  "I'll": "I will",
  "I'll've": "I will have",
  "I'm": "I am",
  "I've": "I have",
  "isn't": "is not",
  "it'd": "it had",
  "it'd've": "it would have",
  "it'll": "it will",
  "it'll've": "it will have",
  "it's": "it is",
  "let's": "let us",
  "ma'am": "madam",
  "mayn't": "may not",
  "might've": "might have",
  "mightn't": "might not",
  "mightn't've": "might not have",
  "must've": "must have",
  "mustn't": "must not",
  "mustn't've": "must not have",
  "needn't": "need not",
  "needn't've": "need not have",
  "o'clock": "of the clock",
  "oughtn't": "ought not",
  "oughtn't've": "ought not have",
  "shan't": "shall not",
  "sha'n't": "shall not",
  "shan't've": "shall not have",
  "she'd": "she would",
  "she'd've": "she would have",
  "she'll": "she will",
  "she'll've": "she will have",
  "she's": "she is",
  "should've": "should have",
  "shouldn't": "should not",
  "shouldn't've": "should not have",
  "so've": "so have",
  "so's": "so is",
  "that'd": "that would",
  "that'd've": "that would have",
  "that's": "that is",
  "there'd": "there had",
  "there'd've": "there would have",
  "there's": "there is",
  "they'd": "they would",
  "they'd've": "they would have",
  "they'll": "they will",
  "they'll've": "they will have",
  "they're": "they are",
  "they've": "they have",
  "to've": "to have",
  "wasn't": "was not",
  "we'd": "we had",
  "we'd've": "we would have",
  "we'll": "we will",
  "we'll've": "we will have",
  "we're": "we are",
  "we've": "we have",
  "weren't": "were not",
  "what'll": "what will",
  "what'll've": "what will have",
  "what're": "what are",
  "what's": "what is",
  "what've": "what have",
  "when's": "when is",
  "when've": "when have",
  "where'd": "where did",
  "where's": "where is",
  "where've": "where have",
  "who'll": "who will",
  "who'll've": "who will have",
  "who's": "who is",
  "who've": "who have",
  "why's": "why is",
  "why've": "why have",
  "will've": "will have",
  "won't": "will not",
  "won't've": "will not have",
  "would've": "would have",
  "wouldn't": "would not",
  "wouldn't've": "would not have",
  "y'all": "you all",
  "y'alls": "you alls",
  "y'all'd": "you all would",
  "y'all'd've": "you all would have",
  "y'all're": "you all are",
  "y'all've": "you all have",
  "you'd": "you had",
  "you'd've": "you would have",
  "you'll": "you you will",
  "you'll've": "you you will have",
  "you're": "you are",
  "you've": "you have"
}


In [38]:
import re

# preprocess sentence
def preprocessing_sentence(sentence, remove_stopwords = True):
    # lowercase
    sentence = sentence.lower()

    # remove parenthesis
    sentence = re.sub(r'\([^)]*\)',' ', sentence)

    # remove double quotes
    sentence = re.sub('"', ' ', sentence) 

    # update contractions
    sentence = ' '.join([contractions[t] if t in contractions else t for t in sentence.split(" ")])
    
    # remove possessive term ('s)
    sentence = re.sub(r"'s\b", " ", sentence)

    # remove special characters
    sentence = re.sub("[^a-zA-Z]", " ", sentence)
    
    # remove stopwords
    if my_stop_word_list:
        tokens = ' '.join(word for word in sentence.split() if not word in my_stop_word_list if len(word)>1)
    else:
        tokens = ' '.join(word for word in sentence.split() if len(word)>1)
    return tokens


In [39]:
# update columns
train['facts'] = [preprocessing_sentence(i) for i in train['facts']]
test['facts'] = [preprocessing_sentence(i) for i in test['facts']]
train['first_party'] = [preprocessing_sentence(i) for i in train['first_party']]
test['first_party'] = [preprocessing_sentence(i) for i in test['first_party']]
train['second_party'] = [preprocessing_sentence(i) for i in train['second_party']]
test['second_party'] = [preprocessing_sentence(i) for i in test['second_party']]

In [40]:
## perform stemming
# train['facts'] = [stem_text_func(i) for i in train['facts']]
# test['facts'] = [stem_text_func(i) for i in test['facts']]

In [41]:
train['facts'] = train['facts'].astype('str')
test['facts'] = test['facts'].astype('str')

In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
def get_vector(vectorizer, df, train_mode):
    if train_mode:
        X_facts = vectorizer.fit_transform(df['facts'])
    else:
        X_facts = vectorizer.transform(df['facts'])
    X_party1 = vectorizer.transform(df['first_party'])
    X_party2 = vectorizer.transform(df['second_party'])
    
    X = np.concatenate([X_party1.todense(), X_party2.todense(), X_facts.todense()], axis=1)
    return X

In [43]:
X = get_vector(vectorizer, train, True)
Y = train["first_party_winner"]
X_test = get_vector(vectorizer, test, False)

In [44]:
X = np.squeeze(np.asarray(X))
X_test = np.squeeze(np.asarray(X_test))

# model train

In [45]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# divide data for validation
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

In [46]:
# load classifiers
from sklearn.linear_model import LogisticRegression, SGDClassifier, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from xgboost import XGBClassifier

# classifiers
names = [
    "Logistic Regression",
    "KNN Classifier",
    "Decision Tree",
    "Linear SVM",
    "Random Forest",
    "SGD Classifier",
    "Ridge Classifier",
    "XGBoost",
    "AdaBoost",
]

models = [
    LogisticRegression(max_iter=1000, random_state=42),
    KNeighborsClassifier(n_neighbors=149),
    DecisionTreeClassifier(random_state=42),
    svm.SVC(kernel = 'linear', random_state=42),
    RandomForestClassifier(n_estimators=250, random_state=42),
    SGDClassifier(loss = 'hinge', random_state=42),
    RidgeClassifier(random_state=42),
    XGBClassifier(random_state=42),
    AdaBoostClassifier(random_state=42),
]

In [47]:
# function to return the summary of baseline models
def score(X_train, y_train, X_val, y_val, names = names, models = models):
    score_df, score_train, score_val = pd.DataFrame(), [], []

    # loop through each model
    for model in models:
        model.fit(X_train, y_train)

        # predict train and validation data
        y_train_pred, y_val_pred = model.predict(X_train), model.predict(X_val)

        # add metrics
        score_train.append(accuracy_score(y_train, y_train_pred))
        score_val.append(accuracy_score(y_val, y_val_pred))
    
    score_df["Classifier"], score_df["Training accuracy"], score_df["Validation accuracy"] = names, score_train, score_val
    score_df.sort_values(by = 'Validation accuracy', ascending = False, inplace = True)
    return score_df

In [48]:
score(X_train, y_train, X_val, y_val, names = names, models = models)

,Classifier,Training accuracy,Validation accuracy
1,KNN Classifier,0.558069,0.596386
3,Linear SVM,1.000000,0.575301
5,SGD Classifier,1.000000,0.572289
6,Ridge Classifier,1.000000,0.569277
0,Logistic Regression,0.989442,0.563253
4,Random Forest,1.000000,0.548193
7,XGBoost,0.999246,0.521084
2,Decision Tree,1.000000,0.496988
8,AdaBoost,0.751885,0.484940


In [49]:
from sklearn.ensemble import VotingClassifier, StackingClassifier

# chose top 4 models, apart from KNN Classifier
clf = StackingClassifier([
    (names[0], models[0]),
    (names[5], models[5]),
    (names[6], models[6]),
    (names[3], models[3]),
])

In [50]:
# quick check of the score
clf.fit(X_train, y_train).score(X_val, y_val)

0.5421686746987951

In [51]:
# re-fit stacking classifier with the whole train dataset
clf.fit(X, Y)

StackingClassifier(estimators=[('Logistic Regression',
                                LogisticRegression(max_iter=1000,
                                                   random_state=42)),
                               ('SGD Classifier',
                                SGDClassifier(random_state=42)),
                               ('Ridge Classifier',
                                RidgeClassifier(random_state=42)),
                               ('Linear SVM',
                                SVC(kernel='linear', random_state=42))])

# predict

In [52]:
sub['first_party_winner'] = clf.predict(X_test)
sub

,ID,first_party_winner
0,TEST_0000,0
1,TEST_0001,0
2,TEST_0002,1
3,TEST_0003,0
4,TEST_0004,1
...,...,...
1235,TEST_1235,0
1236,TEST_1236,1
1237,TEST_1237,1
1238,TEST_1238,0


In [53]:
sub['first_party_winner'].value_counts()

0    638
1    602
Name: first_party_winner, dtype: int64

In [54]:
sub.to_csv('submit.csv', index=False)